In [1]:
import numpy as np

In [6]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def numerical_derivative(f,x):
    delta_x= 1e-4
    grad=np.zeros_like(x) 
    # 수치미분 결과를 grad에 저장
    # x와 똑같은 shape을 가진 배열(혹은 행렬) 생성 (요소값은 0임)
    
    it=np.nditer(x,flags=['multi_index'],op_flags=['readwrite'])
    #모든 입력변수에 대해 편미분하기 위해 iterator 획득
    # iterator 하나 생성. 플래그는 멀티 인덱스 설정
    # 멀티 인덱스를 설정하면 다차원 배열이라도 
    # 순차적으로 interator 생성 후 값을 꺼낼 수 있음
    
    while not it.finished:
        idx=it.multi_index
        
        tmp_val=x[idx] 
        # 임시 변수로 x[idx] 의 원값을 저장
        # numpy 타입은 mutable 이므로 원래 값 보관
        
        x[idx]=float(tmp_val)+delta_x # 전향 차분
        fx1=f(x) #f(x+delta_x) # 첫번째 인자로 들어온 함수를 대입
        
        x[idx]=tmp_val-delta_x # 후향 차분
        fx2=f(x) #f(x-delta_x) # 첫번째 인자로 들어온 함수를 다시 대입
        
        grad[idx]=(fx1-fx2)/(2*delta_x) # 미분 결과를 grad에...
        
        x[idx]=tmp_val
        it.iternext()
        
    return grad

In [21]:
#LogicGate Class
class LogicGate:
    def __init__(self, gate_name, xdata, tdata):
        self.name = gate_name
        
        #입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4, 2) 
        #__xdata에서 언더바 2개 : 밖에서 못 쓰는 것
        # 4 개의 입력데이터 x1,x2에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4, 1) 
        # 4 개의 입력데이터 x1,x2에 대한 각각의 계산 값 행렬
        
        #가중치 W2, 바이어스 b2 초기화
        # 2층 hidden layer unit: 6개 가정
        # 은닉층의 node는 임의의 수(여기선 6개)
        self.__W2 = np.random.rand(2, 6)
        self.__b2 = np.random.rand(6)
        
        #가중치 W3, 바이어스 b3 초기화
        # 3층 output layer unit: 1개
        self.__W3 = np.random.rand(6, 1)
        self.__b3 = np.random.rand(1)
        
        # Deep Learning 규칙 : 앞 layer의 출력과 뒷 layer의 입력 개수를 맞춰야 함
        
        #학습률 learining rate 초기화
        self.__learning_rate = 1e-2
        
        print(self.name+"object is created")
        
    def feed_forward(self): # feed forward를 통하여 손실함수(cross-entropy) 값 계산
        
        delta=1e-7 # log 무한대 발산 방지
        
        z2=np.dot(self.__xdata, self.__W2) + self.__b2 # 은닉층의 선형회귀 값
        a2=sigmoid(z2) # 은닉층의 출력
        
        z3=np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀 값
        y=sigmoid(z3) # 출력층의 출력
        
        # cross-entropy
        return -np.sum(self.__tdata * np.log(y + delta) + (1 - self.__tdata)*np.log((1 - y) + delta))
        
    def loss_val(self):
        
        delta =1e-7
        
        z2=np.dot(self.__xdata, self.__W2) + self.__b2 # 은닉층의 선형회귀 값
        a2=sigmoid(z2) # 은닉층의 출력
        
        z3=np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀 값
        y=sigmoid(z3) # 출력층의 출력
        
        # cross-entropy
        return -np.sum(self.__tdata * np.log(y + delta) + (1 - self.__tdata)*np.log((1 - y) + delta))
        
    
    #수치미분을 이용하여 손실함수가 최소가 될 때까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        print("Initial loss value = ", self.loss_val())
        
        for step in range(16001):
            
            #gradient decent 하기 위한 작업
            # 은닉층, 출력층 함께 작성
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
            
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
            
            #error값이 400의 배수인 것 출력
            if (step % 400 == 0):
                print("Step = ", step, "loss value =", self.loss_val())

    #미래값 예측
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2 #은닉층의 선형회귀 값
        a2 = sigmoid(z2) # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀 값
        y = sigmoid(z3) # 출력층의 출력
    
        if y >= 0.5:
            result = 1  #True
        else:
            result = 0

        return y, result

In [31]:
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,0,0,1])

and_obj=LogicGate('AND',xdata,tdata)

and_obj.train()

ANDobject is created
Initial loss value =  7.735810060436853
Step =  0 loss value = 7.439223216159709
Step =  400 loss value = 2.198987142509861
Step =  800 loss value = 2.0852377083614275
Step =  1200 loss value = 1.9124293213766077
Step =  1600 loss value = 1.656900859194088
Step =  2000 loss value = 1.3343015619058258
Step =  2400 loss value = 1.0034025864229537
Step =  2800 loss value = 0.7286498176076506
Step =  3200 loss value = 0.5345440326198325
Step =  3600 loss value = 0.40392749428585983
Step =  4000 loss value = 0.31479168171239524
Step =  4400 loss value = 0.2522549395859329
Step =  4800 loss value = 0.20709362375461154
Step =  5200 loss value = 0.17357333521083518
Step =  5600 loss value = 0.1480616379437018
Step =  6000 loss value = 0.1282039389727859
Step =  6400 loss value = 0.11243619530333089
Step =  6800 loss value = 0.09969394980430127
Step =  7200 loss value = 0.08923572741831194
Step =  7600 loss value = 0.08053368951848333
Step =  8000 loss value = 0.07320446687

In [32]:
print(and_obj.name, '\n')

test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for data in test_data:
    print(and_obj.predict(data))

AND 

(array([5.3342101e-05]), 0)
(array([0.00602656]), 0)
(array([0.00604345]), 0)
(array([0.98887882]), 1)


In [27]:
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,1,1,1])

or_obj=LogicGate('OR',xdata,tdata)

or_obj.train()

ORobject is created
Initial loss value =  3.0456399526745885
Step =  0 loss value = 3.0227606115682626
Step =  400 loss value = 2.0937173872515173
Step =  800 loss value = 1.9390765614562797
Step =  1200 loss value = 1.6685507411056664
Step =  1600 loss value = 1.3025403145569077
Step =  2000 loss value = 0.9447486040554066
Step =  2400 loss value = 0.6680872982168488
Step =  2800 loss value = 0.4795408961821296
Step =  3200 loss value = 0.35594067884272473
Step =  3600 loss value = 0.2740049561538661
Step =  4000 loss value = 0.21798096334822703
Step =  4400 loss value = 0.17829909172946523
Step =  4800 loss value = 0.14923671037177633
Step =  5200 loss value = 0.12730978346834995
Step =  5600 loss value = 0.11033413280964917
Step =  6000 loss value = 0.09689634646722667
Step =  6400 loss value = 0.08605333858875179
Step =  6800 loss value = 0.07715780164404704
Step =  7200 loss value = 0.06975392667405167
Step =  7600 loss value = 0.06351329020292762
Step =  8000 loss value = 0.05819

In [29]:
print(or_obj.name, '\n')

test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for data in test_data:
    print(or_obj.predict(data))

OR 

(array([0.01073479]), 0)
(array([0.9957972]), 1)
(array([0.99541266]), 1)
(array([0.99969243]), 1)


In [28]:
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([1,1,1,0])

nand_obj=LogicGate('NAND',xdata,tdata)

nand_obj.train()

NANDobject is created
Initial loss value =  3.778816113038485
Step =  0 loss value = 3.7420888424494003
Step =  400 loss value = 2.3626037913118405
Step =  800 loss value = 2.2774542923603844
Step =  1200 loss value = 2.1567051755907722
Step =  1600 loss value = 1.9174499385663482
Step =  2000 loss value = 1.5741090930837456
Step =  2400 loss value = 1.217607770107021
Step =  2800 loss value = 0.9002773095490912
Step =  3200 loss value = 0.6554781750768166
Step =  3600 loss value = 0.4838813874789877
Step =  4000 loss value = 0.36733231841633274
Step =  4400 loss value = 0.2874938821120959
Step =  4800 loss value = 0.2314212903108448
Step =  5200 loss value = 0.19087365316099614
Step =  5600 loss value = 0.1607076338526181
Step =  6000 loss value = 0.1376770343843208
Step =  6400 loss value = 0.1196862980325776
Step =  6800 loss value = 0.10534711853895737
Step =  7200 loss value = 0.09371556920193673
Step =  7600 loss value = 0.08413390023998664
Step =  8000 loss value = 0.07613325992

In [30]:
print(nand_obj.name, '\n')

test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for data in test_data:
    print(nand_obj.predict(data))

NAND 

(array([0.99998114]), 1)
(array([0.99421049]), 1)
(array([0.99421013]), 1)
(array([0.0117708]), 0)


In [22]:
xdata=np.array([[0,0],[0,1],[1,0],[1,1]])
tdata=np.array([0,1,1,0])

xor_obj=LogicGate('XOR',xdata,tdata)

xor_obj.train()

XORobject is created
Initial loss value =  5.410676677432061
Step =  0 loss value = 5.288296179590584
Step =  400 loss value = 2.7615865429479713
Step =  800 loss value = 2.7539112548147053
Step =  1200 loss value = 2.742765461345594
Step =  1600 loss value = 2.7261633155206386
Step =  2000 loss value = 2.7014426616285117
Step =  2400 loss value = 2.665263433616955
Step =  2800 loss value = 2.613832442758609
Step =  3200 loss value = 2.5439040157568575
Step =  3600 loss value = 2.4550202600725393
Step =  4000 loss value = 2.3513319276950684
Step =  4400 loss value = 2.2398897541312426
Step =  4800 loss value = 2.1263845605955263
Step =  5200 loss value = 2.012650685099489
Step =  5600 loss value = 1.8970965459661564
Step =  6000 loss value = 1.7764222447233422
Step =  6400 loss value = 1.6475072669523942
Step =  6800 loss value = 1.509090330337402
Step =  7200 loss value = 1.3627230764417195
Step =  7600 loss value = 1.2126063864518848
Step =  8000 loss value = 1.0645706296928479
Step 

In [24]:
print(xor_obj.name, '\n')

test_data=np.array([[0,0],[0,1],[1,0],[1,1]])

for data in test_data:
    print(xor_obj.predict(data))
    
# 학습량이 부족하면 다른 값이 나올 수 있음 8000 -> 16000으로 증가

XOR 

(array([0.03278547]), 0)
(array([0.96611821]), 1)
(array([0.97259126]), 1)
(array([0.03400035]), 0)
